In [1]:
import pandas as pd
import os
import re
from Secrets.NTE_BMN_sites import sites

In [2]:
file = input("Enter the name of the file: ")

In [3]:
# Read the CSV file into a dataframe
df = pd.read_csv(file)

In [4]:
df = df.rename(columns={'Name': 'Camera Name', 'SITE': 'Building', 'IP': 'IP Address'})
df = df[['Camera Name', 'Building', 'IP Address']]


#### Checking Column Names for Accuracy

In [5]:
correct_cols = [
    "Camera Name",
    "IP Address",
    "Post Deployment Status",
    "Network Fabric",
    "Building",
    "Platform",
]

current_cols = df.columns.tolist()

# Loop through the current columns
for col in current_cols:
    # Check if the column name is in the correct names list
    if col in correct_cols:
        continue
    # Check if a similar named column exists in the correct names list
    for correct_col in correct_cols:
        if col.lower() == correct_col.lower():
            df.rename(columns={col: correct_col}, inplace=True)
            break


#### Checks if Building is present then puts in the correct name

In [6]:
df["Building"] = df["Building"].fillna(df["Camera Name"].str.extract(r"([A-Z]{3}\d{1,3})", expand=False))

#### converting uc_ to "UNDER_CONSTRUCTION" in Camera Names column

In [7]:
df.loc[
    df["Camera Name"].str.startswith("uc_"), "Post Deployment Status"
] = "UNDER_CONSTRUCTION"

In [8]:
df["Camera Name"] = df["Camera Name"].apply(
    lambda x: x.replace("uc_", "") if x.startswith("uc_") else x
)

In [9]:
if 'Platform' not in df.columns:
    df['Platform'] = 'CVR'
else:
    df.loc[df.notna().any(axis=1), 'Platform'] = 'CVR'


#### Clean up the Camera Names column to remove special characters

In [10]:
name_rules = r'[^A-Za-z0-9_\-\.]'

df['Camera Name'] = df['Camera Name'].str.replace(name_rules, '', regex=True)
df['Camera Name'] = df['Camera Name'].str[:64]

#### Keeping only the BMN sites IPs

In [11]:
df = df[df['Building'].isin(sites)]
df = df[df["IP Address"].str.startswith("10.")]
df = df.drop(df[~df["IP Address"].str.startswith("10.")].index)


#### Gets rid of white spaces in the data

In [12]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#### Groups Clusters out of Raw Data sheet

In [13]:
for building_code in df['Building'].str.slice(stop=3).unique():
    
    building_df = df[df['Building'].str.slice(stop=3) == building_code]
    filename = f'{building_code} NTE Cameras.csv'
    building_df.to_csv(filename, index=False)

In [14]:
df.head()

,Camera Name,Building,IP Address,Post Deployment Status,Platform
0,ATL50-RZN-C01-CAGE01-C0111H-W,ATL50,10.105.233.133,NaN,CVR
1,ATL50-RZN-C02-CAGE01-C0504H-W,ATL50,10.105.233.144,NaN,CVR
2,ATL50-RZN-C03-CAGE01-C0803H-NW,ATL50,10.105.233.147,NaN,CVR
3,ATL50-RZN-C04-CAGE01-C0903-H-NW,ATL50,10.105.233.143,NaN,CVR
4,ATL50-RZN-C05-CAGE01-C0901H-NE,ATL50,10.105.233.140,NaN,CVR
